<a href="https://colab.research.google.com/github/DariusTheGeek/Position_2_solution_for_the__African-COVID-19__zindi_hackathon/blob/master/Position_2_(Team_SomeC)_solution_for_the___African_COVID_19__zindi_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

Can we infer important COVID-19 public health risk factors from outdated data? In many countries census and other survey data may be incomplete or out of date. This challenge is to develop a proof-of-concept for how machine learning can help governments more accurately map COVID-19 risk in 2020 using old data, without requiring a new costly, risky, and time-consuming on-the-ground survey.

The 2011 census gives us valuable information for determining who might be most vulnerable to COVID-19 in South Africa. However, the data is nearly 10 years old, and we expect that some key indicators will have changed in that time. Building an up-to-date map showing where the most vulnerable are located will be a key step in responding to the disease. A mapping effort like this requires bringing together many different inputs and tools. For this competition, we’re starting small. Can we infer important risk factors from more readily available data?

The task is to predict the percentage of households that fall into a particularly vulnerable bracket - large households who must leave their homes to fetch water - using 2011 South African census data. Solving this challenge will show that with machine learning it is possible to use easy-to-measure stats to identify areas most at risk even in years when census data is not collected.

## Installing libraries

In [1]:
# Installing the necessary libraries
!pip install catboost
!pip install rgf-python

     |████████████████████████████████| 64.4MB 65kB/s 
     |████████████████████████████████| 798kB 3.3MB/s 


## Importing libraries

In [0]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import requests
from io import StringIO 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, StackingRegressor,HistGradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
import warnings
from rgf.sklearn import RGFRegressor
warnings.filterwarnings('ignore')

## Loading data

In [0]:
# Created links to shared files via google drive
#
train = 'https://drive.google.com/file/d/1_wLi9i-pUk6Kaizjb5i6-Pd0Vi7L1d-E/view?usp=sharing'
test = 'https://drive.google.com/file/d/1OeT53v7tZLnB71n1j4r6KFbtrouz38ej/view?usp=sharing'

# Created a function to read a csv file shared via google and return a dataframe
#
def read_csv(url):
  url = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
  csv_raw = requests.get(url).text
  csv = StringIO(csv_raw)
  df = pd.read_csv(csv)
  return df

# Creating training and testing datataframes
#
train = read_csv(train)
test = read_csv(test)

## Combining the training and test data

In [0]:
# Combining test and train for easy feature engineering.
target = train.target_pct_vunerable

train['separator'] = 0
test['separator'] = 1

train, test = train.align(test, join = 'inner', axis = 1)

comb = pd.concat([train, test])

## Feature engineering

In [0]:
# Examining feature interactions from the most important features from model's feature importances graph and creating new magic features.
# While there is no science into it and it's mostly trial and error, the new features improved the score greatly and if we had computational power, 
# we could have explored more interactions.

comb['household_size'] = comb.total_individuals/comb.total_households
comb['gf_1'] = comb['dw_01'] * comb['psa_01']
comb['gf_2'] = comb['gf_1'] * comb['psa_00']
comb['gf_3'] = comb['gf_1'] * comb['psa_02']
comb['gf_4'] = comb['gf_1'] * comb['psa_03']
comb['gf_5'] = comb['gf_1'] * comb['gf_2']
comb['gf_6'] = comb['gf_5'] * comb['gf_2']
comb['dw_01_2'] = comb['dw_01'] ** 2
comb['psa_00_2'] = comb['psa_00'] ** 2
luxury_stuff = ['psa_01','car_01','stv_00']
not_luxury_stuff = ['psa_00','car_00','stv_01']
comb['luxury_stuff'] = comb[luxury_stuff].sum(axis=1)
comb['not_luxury_stuff'] = comb[not_luxury_stuff].sum(axis=1)
comb['a_luxury_stuff'] = comb[luxury_stuff].mean(axis=1)
comb['a_not_luxury_stuff'] = comb[not_luxury_stuff].mean(axis=1)

## Separating train and test datasets

In [0]:
# Separating the train and test datasets.
train = comb[comb.separator == 0]
test = comb[comb.separator == 1]

train.drop('separator', axis = 1, inplace = True)
test.drop('separator', axis = 1, inplace = True)

## Splitting training and validation sets

In [0]:
# The columns dropped were those that from the feature importance of the baseline model, were of least importance and just added noise to the model.

X = train.drop(columns=['ward', 'dw_13', 'dw_12', 'lan_13', 'psa_03'])
y = target.copy()
tes = test.drop(['ward', 'dw_13', 'dw_12', 'lan_13', 'psa_03'], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2020)

## Training different models

In [0]:
# In stacking, the most important thing is model diversification. from linear, SVM, KNN and Decision trees and many variations of them. 
# The variations are different values of key parameters of each model. 
# While we did not have the time to tune parameters of each model, except the meta learner Catboost, educated guesses on 
# the parameters were made to have as much variability as possible.

estimators_1 = [
    ('xgb', XGBRegressor(random_state=2020, objective ='reg:squarederror', learning_rate=0.05)),
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(random_state=2020)),
    ('lgb', LGBMRegressor(learning_rate=0.2, random_state=2020)),
    ('svr', SVR(degree=2)),
    ('lasso', Lasso(random_state=2020)),
    ('RGF', RGFRegressor()),
    ('kneiba', KNeighborsRegressor(n_neighbors=4)),
    ('cat', CatBoostRegressor(logging_level='Silent', random_state=2020))
]

predictions_1 = StackingRegressor(estimators=estimators_1, final_estimator=CatBoostRegressor(logging_level='Silent', depth=6, bagging_temperature=5, random_state=2020)).fit(X_train, y_train).predict(tes)

estimators_2 = [
    ('xgb', XGBRegressor(objective ='reg:squarederror', learning_rate=0.2, random_state=2020)),
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(random_state=2020)),
    ('lgb', LGBMRegressor(learning_rate=0.05, random_state=2020)),
    ('svr', SVR(degree=5)),
    ('RGF', RGFRegressor()),
    ('lasso', Lasso(random_state=2020)),
    ('kneiba', KNeighborsRegressor(n_neighbors=6)),
    ('cat', CatBoostRegressor(logging_level='Silent', random_state=2020))
]

predictions_2 = StackingRegressor(estimators=estimators_2, final_estimator=CatBoostRegressor(logging_level='Silent', depth=6, bagging_temperature=5, random_state=2020)).fit(X_train, y_train).predict(tes)

predictions_cat_1 = CatBoostRegressor(logging_level='Silent', depth=6, bagging_temperature=5, random_state=2020).fit(X_train, y_train).predict(tes)


# Further averaging, blending and retraining to generalise well
# While the ratios are greater than one, it still works a treat. This is definitely one of the parameters to tune to achieve great results.
stack = [x*0.56 + y*0.51 for x, y in zip(predictions_1, predictions_2)]
stack_2 = [x*0.56 + y*0.51 for x, y in zip(stack, predictions_cat_1)]

X,y = tes.copy(), stack_2
preds_ridge = Ridge(random_state=2020).fit(X, y).predict(X)

# We added a new feature to the test dataset, where we clustered the wards to 150 clusters, then used Catboost's encoder to encode the clusters.
X['cluster'] = KMeans(150, random_state=2020).fit(X).predict(X)
preds_cat = CatBoostRegressor(random_state=2020, verbose = False, depth=6, bagging_temperature=5, cat_features=['cluster']).fit(X, y).predict(X)

# blended the Ridge and Catboost predictions.
final_blend_2 = [x*0.2 +y*0.8 for x, y in zip(preds_ridge, preds_cat)]

# Clipping the values from between 0 - 90 was also important as we know that the target variable is between 0 to 100.
final_blend_2 = np.clip(final_blend_2, a_min=0, a_max=90)

# Applying regularization to the final blend by substracting a constant from the predictions and clipping again.
exp = final_blend_2 - 0.48
exp = np.clip(exp, a_min=0, a_max=90)

## Retraining predictions

In [0]:
# Retraining on the test data by using the prediction of the stacked regressors as our target.
# We also added the clusters but had to manually mean encode the clusters to the target variable as LinearRegression cannot encode categorical variables.
X = tes.copy()

X['cluster'] = KMeans(150, random_state=2020).fit(X).predict(X)
X['target'] = exp
X['encoded'] = X['cluster'].map(X.groupby('cluster')['target'].mean())
y=X.target
X=X.drop(['cluster', 'target'], 1)
preds_1 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.7 + LinearRegression().fit(X, y).predict(X)*0.3
preds_2 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.5 + LinearRegression().fit(X, y).predict(X)*0.5
preds_3 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.6 + LinearRegression().fit(X, y).predict(X)*0.4

final = [x*0.3 + y*0.3 + z*0.4 for x, y, z in zip(preds_1, preds_2, preds_3)]

## Further retraining of predictions

In [0]:
# Retraining again this time using Regularized Greedy Forests and Catboost.
X['final'] = final
y = X.final
X = X.drop('final', 1)
preds_1 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.7 + RGFRegressor().fit(X, y).predict(X)*0.3
preds_2 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.5 + RGFRegressor().fit(X, y).predict(X)*0.5
preds_3 = CatBoostRegressor(verbose = False, random_state=2020).fit(X,y).predict(X)*0.6 + RGFRegressor().fit(X, y).predict(X)*0.4

final2 = [x*0.3 + y*0.3 + z*0.4 for x, y, z in zip(preds_1, preds_2, preds_3)]

## Creating a submission file

In [0]:
# Clipping for the final time and creating the submission file.
final2 = np.clip(final2, a_min=0, a_max=90)
sub_df = pd.DataFrame({'ward': test.ward, 'target_pct_vunerable': final2-0.2})
sub_df.to_csv('submission.csv', index = False)

## Challenges faced

We faced a problem of reproducibility as the score was changing with each submission with no change in code. However, that was solved by setting the *random_state* parameter of all models that have it to the same value. Now, the solution provides a consistently similar score each time it's rerun.


However, the solution has a better private Leader board score of *3.50354986128398* which is better than the score we uploaded in time which was *3.52760301028188*